In [7]:
#Import modules
import os
import pandas as pd
import pybedtools

ModuleNotFoundError: No module named 'pybedtools'

In [5]:
#Function to create a bedtool of relevant regions
def get_regions(feature_scores_table, data_table):
    feature_scores = pd.read_csv(feature_scores_table)
    data = pd.read_csv(data_table)
    print(data[feature_scores.index]['Chromosome', 'Start', 'End'])

In [8]:
!pip install pybedtools

^C
